In [19]:
!pip install pyodbc psycopg2 pandas
!pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.1 MB/s eta 0:00:00


In [20]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
import psycopg2
from io import StringIO

# Azure Blob Storage connection string
azure_connection_string = 'DefaultEndpointsProtocol=https;AccountName=stcommoditygoodcis9440;AccountKey=1ID+03Daliv3RUjzPBEKtBwlppqWuXQVpSToGGDISGeh6yKHD38yISqk8H6TSyGHXdQWDUalCmjt+ASt9p306g==;EndpointSuffix=core.windows.net'

# PostgreSQL connection parameters
pg_host = 'chenchunran.postgres.database.azure.com'
pg_database = 'postgres'
pg_user = 'chenchunran'
pg_password = 'Luck7788'
pg_port = '5432'

# Connect to Azure Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)
container_name = 'commoditygoods'
blob_name = 'commoditygoods_20240512.csv'

# Download the blob as a string
blob_client = blob_service_client.get_blob_client(container_name, blob_name)
blob_data = blob_client.download_blob().readall().decode('utf-8')
print("CSV file downloaded successfully.")

# Load the CSV data into a DataFrame
df = pd.read_csv(StringIO(blob_data))

# Connect to PostgreSQL
pg_conn = psycopg2.connect(
    host=pg_host,
    database=pg_database,
    user=pg_user,
    password=pg_password,
    port=pg_port
)
pg_cursor = pg_conn.cursor()

# Define a function to create the table in PostgreSQL
def create_pg_table(table_name, dataframe):
    # Dynamically create table based on DataFrame columns and dtypes
    columns_with_types = ', '.join([f'{col} TEXT' for col in dataframe.columns])
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        {columns_with_types}
    );
    """
    pg_cursor.execute(create_table_query)
    pg_conn.commit()
    print(f"Table '{table_name}' created successfully.")

# Define a function to insert data into the PostgreSQL table
def insert_data_to_pg(table_name, dataframe):
    insert_query = f"""
    INSERT INTO {table_name} ({', '.join(dataframe.columns)})
    VALUES %s
    """
    data = [tuple(row) for row in dataframe.to_numpy()]
    from psycopg2.extras import execute_values
    execute_values(pg_cursor, insert_query, data)
    pg_conn.commit()
    print(f"Data inserted into '{table_name}' successfully.")

# Define the PostgreSQL table name
pg_table_name = 'commodity_goods'

# Create the PostgreSQL table
create_pg_table(pg_table_name, df)

# Insert the data into the PostgreSQL table
insert_data_to_pg(pg_table_name, df)

# Close PostgreSQL connection
pg_cursor.close()
pg_conn.close()
print("PostgreSQL connection closed.")


CSV file downloaded successfully.
Table 'commodity_goods' created successfully.
Data inserted into 'commodity_goods' successfully.
PostgreSQL connection closed.
